In [1]:
import pandas as pd
from utils import *

data = data_preprocessing('AE_Visit_example20k.csv', with_id=True)



# Get the sum of each patient visit's stay time in ED

In [2]:
sum_time_list = []

def add_stay_tiem(row):
    list = row.values[4:]
    sum_time = 0
    for i in range(0, len(list), 2):
        sum_time += list[i]
    sum_time_list.append(sum_time)
data.apply(add_stay_tiem, axis=1)

1        None
2        None
8        None
10       None
15       None
         ... 
19992    None
19993    None
19994    None
19995    None
20001    None
Length: 10783, dtype: object

In [3]:
sum_time_list

[305.0,
 158.0,
 590.0,
 237.0,
 362.0,
 151.0,
 166.0,
 199.0,
 221.0,
 241.0,
 159.0,
 181.0,
 156.0,
 160.0,
 268.0,
 153.0,
 563.0,
 356.0,
 688.0,
 190.0,
 144.0,
 634.0,
 453.0,
 269.0,
 326.0,
 256.0,
 398.0,
 349.0,
 154.0,
 54.0,
 158.0,
 253.0,
 174.0,
 112.0,
 301.0,
 208.0,
 385.0,
 288.0,
 221.0,
 511.0,
 113.0,
 192.0,
 171.0,
 309.0,
 178.0,
 210.0,
 469.0,
 234.0,
 142.0,
 197.0,
 227.0,
 400.0,
 246.0,
 60.0,
 101.0,
 1.0,
 166.0,
 43.0,
 631.0,
 88.0,
 398.0,
 154.0,
 179.0,
 81.0,
 237.0,
 28.0,
 101.0,
 395.0,
 181.0,
 181.0,
 67.0,
 532.0,
 32.0,
 163.0,
 169.0,
 53.0,
 41.0,
 134.0,
 202.0,
 228.0,
 180.0,
 390.0,
 26.0,
 53.0,
 188.0,
 34.0,
 53.0,
 83.0,
 60.0,
 234.0,
 65.0,
 264.0,
 71.0,
 239.0,
 199.0,
 108.0,
 28.0,
 64.0,
 229.0,
 1.0,
 38.0,
 47.0,
 217.0,
 94.0,
 13.0,
 177.0,
 190.0,
 152.0,
 40.0,
 72.0,
 25.0,
 386.0,
 27.0,
 28.0,
 166.0,
 156.0,
 32.0,
 245.0,
 215.0,
 73.0,
 34.0,
 72.0,
 23.0,
 65.0,
 217.0,
 64.0,
 171.0,
 241.0,
 32.0,
 53.0,
 3

In [4]:
sum_time_series = pd.Series(sum_time_list)
print(sum_time_series.describe())
# calculate the extrame value and remove it
Q1 = sum_time_series.quantile(0.25)
Q3 = sum_time_series.quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
sum_time_series = sum_time_series[sum_time_series < upper_bound]
print(sum_time_series.describe())



count    10783.000000
mean       187.045627
std        243.590220
min          1.000000
25%         96.000000
50%        153.000000
75%        216.000000
max       9741.000000
dtype: float64
count    10150.000000
mean       153.032709
std         81.626039
min          1.000000
25%         93.000000
50%        147.000000
75%        203.000000
max        395.000000
dtype: float64


# Get the stay time of the last area for each admission patient in ED

In [5]:
admission_time_list = []

admission_patient = data[data['isAdmitted'] == 1]

def add_last_stay_time(row):
    list = row.values[4:]
    n = 0
    for i in range(0, len(list), 2):
        if list[i] == 0:
            n = i
            break
    admission_time_list.append(list[n-2])

admission_patient.apply(add_last_stay_time, axis=1)
admission_time_series = pd.Series(admission_time_list)
print(admission_time_series.describe())

count    1991.000000
mean      232.556504
std       157.689574
min         2.000000
25%       137.000000
50%       199.000000
75%       291.000000
max      1551.000000
dtype: float64


In [6]:
Q1 = admission_time_series.quantile(0.25)
Q3 = admission_time_series.quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
admission_time_series = admission_time_series[admission_time_series < upper_bound]
print(admission_time_series.describe())

count    1880.000000
mean      205.865957
std       109.749869
min         2.000000
25%       132.000000
50%       191.500000
75%       264.000000
max       513.000000
dtype: float64


In [8]:
import joblib
import numpy as np
import torch
import datetime
import pandas as pd

from lstm_model import LSTM

def get_patient_data(patients, time, id, extractor, classifier, device = 'cuda'):
    patient = patients[patients['SpellID_Anon'] == id]
    patient.loc[:, 'FirstTimetoED'] = pd.to_datetime(
        patient['FirstTimetoED'], format='%Y-%m-%d %H:%M:%S')
    first_time = patient['FirstTimetoED'].values[0]
    time_min_d = (first_time-time).total_seconds() / 60
    patient.loc[:, 'FirstTimetoED'] = first_time.hour + first_time.minute/60
    patient = patient.values[0, 1:]
    time_sum = 0
    for i in range(3, len(patient), 2):
        time_sum += patient[i]
        if time_sum >= time_min_d:
            patient[i] -= (time_sum - time_min_d)
            patient[i+1:] = 0
            break
    patient = np.array(patient, dtype=np.float32)
    region_sequences = torch.tensor(patient[2:-1:2], dtype=torch.int).to(device)
    time_sequences = torch.tensor(patient[3::2], dtype=torch.float).to(device)
    other_features = patient[:2]
    extractor.eval()
    with torch.no_grad():
        region_embed = extractor.region_embedding(region_sequences)
        time_embed = extractor.time_embedding(
            region_sequences) * time_sequences.unsqueeze(-1)
        combined_embed = torch.cat([region_embed, time_embed], dim=-1)
        _, (hidden, _) = extractor.lstm(combined_embed.float())
        extracted_features = hidden.squeeze(0).cpu().numpy()
    extracted_features = np.concatenate(
        [other_features, extracted_features])
    extracted_features = extracted_features.reshape(1, -1)
    predict_proba = classifier.predict_proba(
        extracted_features)[0][1]
    temp = {
        'id': id,
        'predict_proba': predict_proba
    }
    return temp

In [9]:
pred_beds_list = []
act_beds_list = []
time = pd.to_datetime('2019-06-01 00:00:00')
data = pd.read_csv('AE_Visit_example20k.csv', low_memory=False)
patients = pd.read_csv('patients_with_id.csv', low_memory=False)
data = data[data['SpellID_Anon'].isin(patients['SpellID_Anon'])]
data['StartTime'] = pd.to_datetime(
        data['StartTime'], format='%d/%m/%Y %H:%M')
data['EndTime'] = pd.to_datetime(data['EndTime'], format='%d/%m/%Y %H:%M')
params = {'hidden_size': 64, 'lr': 0.001, 'region_embedding_dim': 16,
          'time_embedding_dim': 32, 'batch_size': 20000}
extractor = LSTM(vocab_size=20, region_embedding_dim=params['region_embedding_dim'],
            time_embedding_dim=params['time_embedding_dim'], hidden_size=params['hidden_size'], output_size=3).to('cuda')
extractor.load_state_dict(torch.load('models/lstm_xgb/lstm.pth'))
classifier = joblib.load('models/lstm_xgb/xgb.dta')

In [ ]:
def get_interval_data(interval, hours=1):
    now_patients = data[(data['StartTime'] <= interval) & (data['EndTime'] >= interval)]
    ids = now_patients['SpellID_Anon'].values
    temp_list = [get_patient_data(patients, interval, id, extractor, classifier) for id in ids]
    pred_beds = sum(temp['predict_proba'] for temp in temp_list)
    # hours in the future
    end_time = interval + datetime.timedelta(hours=hours)
    future_patients = data[(data['EndTime'] <= end_time) & (data['EndTime'] >= interval)]
    transfer_status_list = pd.Series(future_patients['TransferStatus'].unique())
    uch_list = transfer_status_list[~transfer_status_list.isin(
        ['Arrival-Discharge', 'Arrival', 'Discharged', 'Transfer'])]
    future_patients_admitted_list = future_patients[future_patients['TransferStatus'].isin(uch_list)]
    act_beds = len(future_patients_admitted_list)
    return pred_beds, act_beds

In [ ]:
day = 30
hour = 24 * day
time_intervals = [time + datetime.timedelta(hours=n) for n in range(0, hour, 1)]
pred_beds_list, act_beds_list = zip(*[get_interval_data(interval) for interval in time_intervals])

In [ ]:
import matplotlib.pyplot as plt
pred_beds_list = np.ceil(pred_beds_list)
plt.figure(figsize=(40, 10))
plt.plot(pred_beds_list, label='Predicted Beds')
plt.plot(act_beds_list, label='Actual Beds')
plt.xlabel(f'Number of Hours (Start from {time})')
plt.ylabel('Beds')
plt.title(f'Predicted and Actual Beds for the next 1 hour ({day} Days)')
plt.legend()
plt.show()

In [ ]:
import numpy as np
from scipy.stats import spearmanr

# act_beds_list, pred_beds_list 是你的数据
corr, _ = spearmanr(act_beds_list, pred_beds_list)

print(f"Spearman Rank Correlation Coefficient: {corr}")

In [ ]:
import numpy as np
from scipy.stats import linregress

# act_beds_list, pred_beds_list 是你的数据
slope, intercept, r_value, p_value, std_err = linregress(act_beds_list, pred_beds_list)

# CCC 是 r_value 的平方
ccc = r_value**2

print(f"Concordance Correlation Coefficient: {ccc}")

In [ ]:
# Pearson Correlation Coefficient
correlation_matrix = np.corrcoef(act_beds_list, pred_beds_list)

# Pearson Correlation Coefficient 是矩阵的非对角线元素
pearson_correlation_coefficient = correlation_matrix[0, 1]

print(f"Pearson Correlation Coefficient: {pearson_correlation_coefficient}")

In [ ]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(act_beds_list, pred_beds_list)
mae

In [ ]:
from scipy import spatial
cos_sim = 1 - spatial.distance.cosine(act_beds_list, pred_beds_list)
cos_sim